# 情感极性

In [1]:
import pandas as pd
import jieba
df = pd.read_excel("E:/comorbidity/data/final_data.xlsx")
# 加载情感词典（从文件加载）
def load_word_list(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        words = set([line.strip() for line in f.readlines()])
    return words

# 假设文件路径是相对路径或绝对路径，替换为你的实际文件路径
positive_words_path = "E:/comorbidity/feature_extraction/情感情绪/情感极性词典/正面情绪词.txt"  # 替换为正向词典文件路径
negative_words_path ="E:/comorbidity/feature_extraction/情感情绪/情感极性词典/负面情绪词.txt"  # 替换为负向词典文件路径

# 加载正向和负向情感词汇
positive_words = load_word_list(positive_words_path)
negative_words = load_word_list(negative_words_path)

# 手动添加正向和负向情感词汇
manual_positive_words = {"好开心", "高兴", "喜欢", "愉快", "热爱", "祝福", "美好",'开森','快乐','希望',
                        '好看','花花','快乐','分享','落日','真好','音乐','日常','灿烂','礼物','顺遂',
                        '太阳','好吃','风景','粉嫩','小猫咪','美文','好事','加油'}
manual_negative_words = {"失望", "难过", "痛苦", "悲伤", "不满", "愤怒", "沮丧","药","喉咙","干呕","自卑","懦弱","住院",
                        "抑郁","崩溃","倾诉","严重","药物","好累","压力","不开心",'焦虑','吵','睡不着','抱抱','失望',
                        '淡淡的','好累','睡不着','医院','怀疑','烂透','犯病','鼠掉','好累','封闭','噩梦','活着',
                        '离职','胸闷','死去','躯体化','想死','心慌','又是','失眠','想哭','抱抱','倒霉','好难','百般刁难',
                        '糟糕','我恨','孤独','好难','气死我了','寻求','双相','轻度','得病','吃药','睡不着','想哭'}

# 将手动词汇添加到词典中
positive_words.update(manual_positive_words)  # 将手动正向词汇加入正向情感词典
negative_words.update(manual_negative_words)  # 将手动负向词汇加入负向情感词典

# 自定义分词和情感分析函数
def sentiment_analysis(text):
    if not text:  # 检查文本是否为空
        return '中性'

    words = jieba.lcut(str(text))
    pos_count = len([w for w in words if w in positive_words])
    neg_count = len([w for w in words if w in negative_words])
    
    if pos_count > neg_count:
        return '正向'
    elif neg_count > pos_count:
        return '负向'
    else:
        return '中性'


# 应用情感分析
df['BOSON情感分析结果'] = df['分词文本'].apply(sentiment_analysis)

# 只保存序号和情感分析结果
result_df = df[['序号', 'BOSON情感分析结果']]

# 保存为 Excel 文件
output_path = 'E:/comorbidity/feature_extraction/情感情绪/BOSON情感分析结果.xlsx'  # 输出的 Excel 文件路径
result_df.to_excel(output_path, index=False)

print(f"情感分析结果已经成功保存为 {output_path}")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\qy201\AppData\Local\Temp\jieba.cache
Loading model cost 0.464 seconds.
Prefix dict has been built successfully.


情感分析结果已经成功保存为 E:/comorbidity/feature_extraction/情感情绪/BOSON情感分析结果.xlsx


In [2]:
# 查看每种情感分析结果的出现次数
sentiment_counts = df['BOSON情感分析结果'].value_counts()

# 打印结果
print(sentiment_counts)


BOSON情感分析结果
负向    48380
正向    23441
中性    23342
Name: count, dtype: int64


In [ ]:
'''
from collections import Counter

filtered_df = df[(df['auto_label'].isin(['抑郁', '焦虑', '抑郁焦虑'])) & (df['BOSON情感分析结果'] == '正向')]
# 将筛选后的分词文本合并为一个长字符串
all_text = ' '.join(filtered_df['分词文本'].dropna().astype(str))  # 合并文本并去掉空值

# 将合并的文本按空格分割成单词列表
words = all_text.split()  # 使用空格分隔词语

# 统计词频
word_counts = Counter(words)

# 将词频结果转换为 DataFrame
word_freq_df = pd.DataFrame(word_counts.items(), columns=['词语', '频次'])

# 按频次降序排序
word_freq_df = word_freq_df.sort_values(by='频次', ascending=False)

# 打印前100个高频词及其频次
print("前100个高频词及其频率：")
print(word_freq_df.head(100))

'''

前100个高频词及其频率：
        词语    频次
1079   抑郁症  4302
6       没有  3765
189     真的  2409
104     感觉  2171
98      可以  2095
...    ...   ...
22579   ##   402
11      不好   398
1012    最后   388
483     每次   388
1542    容易   386

[100 rows x 2 columns]


In [3]:

##snownlp效率低下，所以这部分特征不用
import pandas as pd
from snownlp import SnowNLP

# 定义情感分析函数，返回情感分数
def analyze_sentiment_score(text):
    if pd.isna(text) or str(text).strip() == '':  # 处理空值
        return None  # 空文本返回 None
    s = SnowNLP(str(text))
    score = s.sentiments  # 获取情感分数
    return score

# 应用情感分析，返回情感分数
df['情感分析分数'] = df['发帖内容'].apply(analyze_sentiment_score)

# 查看结果
print(df[['发帖内容', '情感分析分数']])


                                                    发帖内容    情感分析分数
0      我不知道我是怎么了。一开始玩王者的时候，觉得这只是个游戏而已没必要太在乎输赢，开心就好，当时...  0.799442
1                                               心情糟糕，晚安吧  0.448440
2                                     我真的好想死啊，我活不下去了，好痛苦  0.656004
3      B站真的是太懂我了，我刷到这条视频，说的是报复性熬夜，视频说的是白天一直上班，到了晚上就开始...  1.000000
4                      明天就放假啦，同担买了后天的高铁票来找我玩，没有好朋友我可怎么办啊  0.077196
...                                                  ...       ...
95158        请问每次紧张就会喉咙发紧然后一直吞口水，然后就会有要吐干呕的感觉应该怎么缓解或者解决呢  0.031647
95159  在跟朋友跨年，现在还没回家，爸爸打电话🤳，我没接，然后打给姑姑，我刚刚回了，对爸爸的态度不太...  0.032254
95160             宝儿们请问紧张就会喉咙发紧一直吞口水该怎么缓解啊，我一直这样就会觉得有点想呕  0.602074
95161                                    2024 要勇敢和这里说再见！  0.820228
95162                             各位病友，新年快乐，新一年要多鼓励一下自己。  0.957001

[95163 rows x 2 columns]


In [4]:
# 只保存序号和情感分析结果
result_df = df[['序号', '情感分析分数']]

# 保存为 Excel 文件
output_path = 'E:/comorbidity/feature_extraction/情感情绪/SnowNLP情感分析分数.xlsx'  # 输出的 Excel 文件路径
result_df.to_excel(output_path, index=False)

print(f"情感分析结果已经成功保存为 {output_path}")

情感分析结果已经成功保存为 E:/comorbidity/feature_extraction/情感情绪/SnowNLP情感分析分数.xlsx


情感强度

In [6]:

###情感强度计算示例
import re

# 定义负面情绪极端表达的关键词列表
negative_keywords = [
    "完全无法忍受", "彻底失败", "极其痛苦", "绝望至极", "生不如死", "无法承受"
]

# 文本预处理函数
def preprocess_text(text):
    # 去除标点符号
    text = re.sub(r'[^\w\s]', '', text)
    # 转换为小写字母
    text = text.lower()
    return text

# 计算情绪强度函数
def calculate_emotion_intensity(text):
    preprocessed_text = preprocess_text(text)
    intensity = 0
    for keyword in negative_keywords:
        if keyword in preprocessed_text:
            intensity += 1
    return intensity

# 示例文本
text = "我感觉完全无法忍受这种情况，这简直是彻底失败。"

# 计算情感强度
emotion_intensity = calculate_emotion_intensity(text)
print(f"情感强度: {emotion_intensity}")

情感强度: 2


In [7]:
##负面关键词不赋予权重
import re
import csv
from tqdm import tqdm
# 定义负面情绪极端表达的关键词列表
negative_keywords = [

       # "完全无法忍受", "彻底失败", "极其痛苦", "绝望至极", "生不如死", "无法承受",''
    '很', "非常", "真的", "简直", "绝", "无法",'彻底','死','最','不','又','哭','没有','崩溃','药','自杀','割腕','超级','活','绝望',
    '严重','孤独','葬'
]

# 文本预处理函数
def preprocess_text(text):
    # 去除标点符号
    text = re.sub(r'[^\w\s]', '', text)
    # 转换为小写字母
    text = text.lower()
    return text

# 计算情绪强度函数
def calculate_emotion_intensity(text):
    preprocessed_text = preprocess_text(text)
    intensity = 0
    for keyword in negative_keywords:
        if keyword in preprocessed_text:
            intensity += 1
    return intensity

# 存储结果的列表
results = []

# 从txt文件中读取文本，每行作为一条记录
with open("E:/comorbidity/data/output.txt", 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 分别计算每条记录的情绪强度，并保存到结果列表中
for index, line in enumerate(lines, 1):
    emotion_intensity = calculate_emotion_intensity(line.strip())
    results.append({
        "文档序号": index,
        "情感强度": emotion_intensity
    })

# 将结果写入csv文件
with open('E:/comorbidity/feature_extraction/情感情绪/情感强度.csv', 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ["文档序号","情感强度"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

print("已成功将结果导出到情感强度.csv 文件中。")

已成功将结果导出到情感强度.csv 文件中。
